In [ ]:
from saf.preprocessing_functions import mask_and_blur_annulus
from saf.ratchet_model import calculate_offsets
from saf.filters import n_fold_filter, annulus_filter
import numpy as np
import math
import matplotlib.pyplot as plt

The numors are:
56358, 56059, 55760, 55461, 55162, 54863, 54261

In [ ]:
#data_dir = '/Users/cadenmyers/billingelab/dev/sym_adapted_filts/data/SkL_data/npz_sept_numor_data/'
data_dir = '/Users/nathanchalus/Desktop/PSI Cu2OSeO3 Corbino September 2024/Analysis/ML Analysis/saf/data/SkL_data/calculated_offsets/npz_sep_data/'
numor = 56358
dps = np.load(data_dir + f'{numor}.npz')['data']
dps_masked = mask_and_blur_annulus(dps, blur=False, inner_radius=17)
plt.figure(figsize=(5, 5))
plt.imshow(dps_masked[0])

In [ ]:
# determine r value for the masking
best_rs = []

for i in range(0, 299):
    best_r_0 = None
    max_sum = -np.inf
    for r_0 in np.arange(20, 40, .5):  # Iterate through a range of r_0 values
        a = annulus_filter(r_0=r_0, sd=3, cx=64, cy=62, imshape=dps[i].shape)
        current_sum = (dps[i] * a).sum()
        if current_sum > max_sum:
            max_sum = current_sum
            best_r_0 = r_0
    best_rs.append(best_r_0)
    # print(f"{i}: Best r_0: {best_r_0}, Maximum sum: {max_sum}")

average_best_r_0 = np.mean(best_rs)
print(f"Average of best_r_0 values: {average_best_r_0}")
a = annulus_filter(r_0=average_best_r_0, sd=3.5, cx=64, cy=62, imshape=dps[i].shape)
a_thresholded = np.where(a < 0.1, 0, 1)

fig, ax = plt.subplots(2, 2, figsize=(10, 10))
fontsize = 15
ax[0, 0].imshow(a)
ax[0, 0].text(5, 7, 'a', color='white', fontsize=fontsize, fontweight='bold')

ax[0, 1].imshow(a_thresholded)
ax[0, 1].text(5, 7, 'b', color='white', fontsize=fontsize, fontweight='bold')

ax[1, 0].imshow(dps[i])
ax[1, 0].text(5, 7, 'c', color='white', fontsize=fontsize, fontweight='bold')

ax[1, 1].imshow(dps[i] * a_thresholded)
ax[1, 1].text(5, 7, 'd', color='white', fontsize=fontsize, fontweight='bold')

for axis in ax.flatten():
    axis.axis('off')
plt.tight_layout()
plt.savefig('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/figures/mask_dps.pdf')
plt.show()


In [ ]:
# Calculate offsets for for non-blurred data
from matplotlib.cm import viridis
from matplotlib.colors import Normalize

numors = [56358, 56059, 55760, 55461, 55162, 54863, 54261]
#numors = [56358]

norm = Normalize(vmin=min(numors), vmax=max(numors))  # Normalize numors to [0, 1]

def get_orientation(numors, blur):
    offset1s = []
    offset2s = []
    overlap1s = []
    overlap2s = []
    for numor in numors:
        dps = np.load(data_dir + f'{numor}.npz')['data']
        dps_masked = mask_and_blur_annulus(dps, blur=blur)
        o1, o2, ov1, ov2 = calculate_offsets(
            dps_masked, n_folds=6, k=8.55, tolerance_forward=15, tolerance_reverse=3
        )
        offset1s.append(o1)
        offset2s.append(o2)
        overlap1s.append(ov1)
        overlap2s.append(ov2)
        color = viridis(norm(numor))
        plt.plot(o1, label=f'numor {numor}', color=color)

    plt.legend()
    plt.show()
    return offset1s, offset2s, overlap1s, overlap2s

offset1raw, offset2raw, overlap1, overlap2 = get_orientation(numors, blur=False)

# offset1blur, offset2blur = get_orientation(numors, blur=True)

In [ ]:
# --- per-numor plots + collect ratios ---
avg_ratios = []
for i, numor in enumerate(numors):
    o1 = np.array(overlap1[i])
    o2 = np.array(overlap2[i])
    
    # individual overlap scores (negated if you prefer)
    plt.figure()
    plt.plot(-o1, label='Domain 1')
    plt.plot(-o2, label='Domain 2')
    ratio = o1.mean() / o2.mean()
    avg_ratios.append(ratio)
    
    # annotate the per-numor average ratio
    plt.text(
        0.05, 0.95,
        f'Avg overlap ratio: {ratio:.2f}',
        transform=plt.gca().transAxes,
        fontsize=12, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.7)
    )
    plt.legend(loc='lower right')
    plt.title(f'Numor {numor}')
    plt.xlabel('Frame')
    plt.ylabel('Overlap score')
    plt.grid(False)
    plt.box(True)
    plt.show()

# --- summary plot of all ratios ---
mean_ratio = np.mean(avg_ratios)
std_ratio  = np.std(avg_ratios)

plt.figure(figsize=(8, 5))
plt.plot(numors, avg_ratios, marker='o', linestyle='-', label='Avg ratio per numor')

# mean line and ±1 std band
plt.axhline(mean_ratio, color='red', linestyle='--', label=f'Mean = {mean_ratio:.2f}')
plt.fill_between(
    numors,
    mean_ratio - std_ratio,
    mean_ratio + std_ratio,
    color='red', alpha=0.2,
    label=f'±1 STD = {std_ratio:.2f}'
)

# textbox with stats
plt.text(
    0.05, 0.95,
    f'Overall mean: {mean_ratio:.2f}\nOverall std:  {std_ratio:.2f}',
    transform=plt.gca().transAxes,
    fontsize=12, verticalalignment='top',
    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8)
)

plt.xlabel('Numor')
plt.ylabel('Mean(Overlap₁) / Mean(Overlap₂)')
plt.title('Summary of Overlap Ratios Across Numors')
plt.legend(loc='lower right')
plt.grid(False)
plt.box(True)
plt.tight_layout()
plt.show()

In [ ]:
f = n_fold_filter(k=8.55, offset=0, n_folds=6, imshape=(128, 128))
plt.imshow(f)


In [ ]:

def intensity_vs_phi(f, r_values=np.arange(5, 65, 5), center=None, n_phi=360):
    """Plots intensity vs angle (phi) for specified radii in a 2D image."""
    if center is None:
        center = (f.shape[0] // 2, f.shape[1] // 2)
    if r_values is None:
        r_values = np.arange(5, min(f.shape) // 2, 5)

    phis = np.linspace(0, 2 * np.pi, n_phi, endpoint=False)

    for r in r_values:
        x = center[1] + r * np.cos(phis)
        y = center[0] + r * np.sin(phis)
        intensities = f[y.astype(int), x.astype(int)]
        plt.plot(np.degrees(phis), intensities+r/10, label=f"r = {r}")

    plt.xlabel("Angle (°)")
    plt.ylabel("Intensity")
    plt.title("Intensity vs. Angle at Various Radii")
    plt.legend()
    plt.show()

intensity_vs_phi(f)


In [ ]:
# plot blurred and non-blurred offsets

fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Plot blurred and non-blurred offsets
for i, numor in enumerate(numors):
    axs[0].plot(offset1blur[i], label=f'smoothed {numor}', color=viridis(norm(numor)), linestyle='--')
    axs[0].plot(offset1raw[i], label=f'raw {numor}', alpha=.5, color=viridis(norm(numor)))
axs[0].set_title("Blurred and Non-Blurred Offsets")
axs[0].legend()
axs[0].set_ylabel("Degrees")

# Plot difference curve
for i, numor in enumerate(numors):
    axs[1].plot(np.array(offset1blur[i]) - np.array(offset1raw[i]), label=f'difference {numor}', color=viridis(norm(numor)), linestyle=':')
axs[1].set_title("Difference Curve")
axs[1].legend()

plt.show()

    # plt.plot(smoothed_offset1 - offset1s[i], label=f'difference numor {numor}', color=viridis(norm(numor)), linestyle=':')

In [ ]:
# determine r value for the masking
best_rs = []

for i in range(0, 299):
    best_r_0 = None
    max_sum = -np.inf
    for r_0 in np.arange(20, 40, .5):  # Iterate through a range of r_0 values
        a = annulus_filter(r_0=r_0, sd=3, cx=64, cy=62, imshape=dps[i].shape)
        current_sum = (dps[i] * a).sum()
        if current_sum > max_sum:
            max_sum = current_sum
            best_r_0 = r_0
    best_rs.append(best_r_0)
    print(f"{i}: Best r_0: {best_r_0}, Maximum sum: {max_sum}")

average_best_r_0 = np.mean(best_rs)
print(f"Average of best_r_0 values: {average_best_r_0}")
a = annulus_filter(r_0=average_best_r_0, sd=3.5, cx=64, cy=62, imshape=dps[i].shape)
a_thresholded = np.where(a < 0.1, 0, 1)

fig, ax = plt.subplots(2, 2, figsize=(10, 10))
fontsize = 15
ax[0, 0].imshow(a)
ax[0, 0].text(5, 7, 'a', color='white', fontsize=fontsize, fontweight='bold')

ax[0, 1].imshow(a_thresholded)
ax[0, 1].text(5, 7, 'b', color='white', fontsize=fontsize, fontweight='bold')

ax[1, 0].imshow(dps[i])
ax[1, 0].text(5, 7, 'c', color='white', fontsize=fontsize, fontweight='bold')

ax[1, 1].imshow(dps[i] * a_thresholded)
ax[1, 1].text(5, 7, 'd', color='white', fontsize=fontsize, fontweight='bold')

for axis in ax.flatten():
    axis.axis('off')
plt.tight_layout()
plt.savefig('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/figures/mask_dps.pdf')
plt.show()
